In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from bs4 import BeautifulSoup
import lxml
import requests
import unicodedata

In [2]:
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
warnings.simplefilter(action='ignore', category=(FutureWarning))

In [3]:
fm_df = pd.read_csv("players_db/fm23/uluta_fm23data.csv")

In [4]:
fm_df.columns = [col.strip() for col in fm_df.columns]

for col in fm_df.select_dtypes("object").columns:
   fm_df[col] = fm_df[col].apply(lambda x: x.strip())

In [5]:
# Assume df is your DataFrame
new_column_names = {
    'Acc': 'Acceleration',
    'Aer': 'Aerial Reach',
    'Agg': 'Aggression',
    'Agi': 'Agility',
    'Ant': 'Anticipation',
    'Bal': 'Balance',
    'Bra': 'Bravery',
    'Cmd': 'Command of Area',
    'Com': 'Communication',
    'Cmp': 'Composure',
    'Cnt': 'Concentration',
    'Cor': 'Corners',
    'Cro': 'Crossing',
    'Dec': 'Decisions',
    'Det': 'Determination',
    'Dri': 'Dribbling',
    'Ecc': 'Eccentricity',
    'Fin': 'Finishing',
    'Fir': 'First Touch',
    'Fla': 'Flair',
    'Fre': 'Free Kick Taking',
    'Han': 'Handling',
    'Hea': 'Heading',
    'Jum': 'Jumping Reach',
    'Kic': 'Kicking',
    'Ldr': 'Leadership',
    'Lon': 'Long Shots',
    'L Th': 'Long Throws',
    'Mar': 'Marking',
    'Nat .1': 'Fitness',
    'OtB': 'Off the Ball',
    '1v1': 'One vs One',
    'Pac': 'Pace',
    'Pas': 'Passing',
    'Pen': 'Penalty Taking',
    'Pos': 'Positioning',
    'Pun': 'Punching',
    'Ref': 'Reflexes',
    'TRO': 'Rushing Out',
    'Sta': 'Stamina',
    'Str': 'Strength',
    'Tck': 'Tackling',
    'Tea': 'Teamwork',
    'Tec': 'Technique',
    'Thr': 'Throwing',
    'Vis': 'Vision',
    'Wor': 'Work Rate'
}

# new_column_names = {key: value.lower() for key, value in new_column_names.items()}
fm_df.rename(columns=new_column_names, inplace=True)

In [10]:
fm_df.query("Name == 'Joshua Kimmich'").iloc[0].to_dict()

{'UID': 92039023,
 'Inf': '',
 'Name': 'Joshua Kimmich',
 'DoB': '8/2/1995 (27 years old)',
 'Nat': 'GER',
 'Division': 'Bundesliga',
 'Club': 'FC Bayern',
 'Based': 'Germany (Bundesliga)',
 'Preferred Foot': 'Right',
 'Right Foot': 'Very Strong',
 'Left Foot': 'Fairly Strong',
 'Position': 'D/WB (R), DM, M (C)',
 'Height': '177 cm',
 'Weight': '75 kg',
 'Age': 27,
 'Transfer Value': '€96M - €119M',
 'Wage': '€1,496,000 p/m',
 'AT Apps': '253',
 'AT Gls': '23',
 'Team': '',
 'Caps': 68,
 'Yth Apps': '14',
 'Style': 'Leader',
 'Rc Injury': '-',
 'Best Role': 'Deep Lying Playmaker',
 'Best Duty': 'Defend',
 'Best Pos': 'DM',
 'Acceleration': 13,
 'Aerial Reach': 1,
 'Aggression': 15,
 'Agility': 12,
 'Anticipation': 15,
 'Balance': 13,
 'Bravery': 15,
 'Command of Area': 2,
 'Communication': 2,
 'Composure': 17,
 'Concentration': 14,
 'Corners': 15,
 'Crossing': 17,
 'Decisions': 16,
 'Determination': 20,
 'Dribbling': 12,
 'Eccentricity': 3,
 'Finishing': 11,
 'First Touch': 15,
 'Flair

In [236]:
fm_attributes = list(new_column_names.values())
gk_attributes = [ "Command of Area", "Communication", "First Touch", "Handling", "Kicking", "One vs One", "Punching", "Reflexes", "Rushing Out", "Throwing" ]
gk_technical_attributes = [ "Technique" ] # "Free Kick Taking", "Penalty Taking", 
mental_attributes = [ "Aggression", "Anticipation", "Bravery", "Composure", "Concentration", "Decisions", "Determination", "Flair", "Leadership", "Off the Ball", "Positioning", "Teamwork", "Vision", "Work Rate" ]
physical_attributes = [ "Acceleration", "Agility", "Balance", "Jumping Reach", "Fitness", "Pace", "Stamina", "Strength" ]
technical_attributes = [ "Corners", "Crossing", "Dribbling", "Finishing", "First Touch", "Free Kick Taking", "Heading", "Long Shots", "Long Throws", "Marking", "Passing", "Penalty Taking", "Tackling", "Technique" ]

fm_df[fm_attributes] = fm_df[fm_attributes] * 4.95

In [237]:
fm_df.head(n=5)

UID  Inf               Name                        DoB  Nat  \
0    831079       Giorgio Chiellini   14/8/1984 (37 years old)  ITA   
1   2105510                    Hulk   25/7/1986 (35 years old)  BRA   
2  78000335             Luis Suárez   24/1/1987 (35 years old)  URU   
3    962988  Ret    Gonzalo Higuaín  10/12/1987 (34 years old)  ARG   
4  19007730           Alex Teixeira    6/1/1990 (32 years old)  BRA   

                             Division         Club                     Based  \
0                 Major League Soccer         LAFC              U.S.A. (MLS)   
1   Brazilian National First Division          ATM   Brazil (First Division)   
2            Uruguayan First Division     Nacional  Uruguay (First Division)   
3                 Major League Soccer  Inter Miami              U.S.A. (MLS)   
4  Brazilian National Second Division          VDG  Brazil (Second Division)   

  Preferred Foot   Right Foot  ... Reflexes Rushing Out Stamina Strength  \
0      Left Only         Weak  ...     4.95        9.90   49.50    69.30   
1           Left   Reasonable  ...     9.90        4.95   74.25    89.10   
2          Right  Very Strong  ...     4.95       14.85   49.50    69.30   
3          Right  Very Strong  ...     4.95       14.85   49.50    79.20   
4          Right  Very Strong  ...     4.95       14.85   54.45    34.65   

   Tackling Teamwork Technique Throwing Vision Work Rate  
0     89.10    84.15     39.60    19.80  34.65     74.25  
1     39.60    39.60     74.25    14.85  69.30     54.45  
2     44.55    64.35     74.25    14.85  74.25     44.55  
3     39.60    49.50     79.20     4.95  69.30     49.50  
4     24.75    49.50     79.20     9.90  54.45     49.50  

[5 rows x 74 columns]

In [238]:
fm_df['DoB'][0]

'14/8/1984 (37 years old)'

In [239]:
convert_date_format = lambda x: pd.to_datetime(x, format='%d/%m/%Y').strftime('%Y-%m-%d')
fm_df['DoB'] = fm_df['DoB'].apply(lambda x: x.split(" ")[0])
fm_df['DoB'] = fm_df['DoB'].apply(convert_date_format)

In [240]:
fm_df['DoB'][0]

'1984-08-14'

In [241]:
fm_df['Height'] = fm_df['Height'].apply(lambda x: x.split(" ")[0])

In [242]:
fm_df.columns

Index(['UID', 'Inf', 'Name', 'DoB', 'Nat', 'Division', 'Club', 'Based',
       'Preferred Foot', 'Right Foot', 'Left Foot', 'Position', 'Height',
       'Weight', 'Age', 'Transfer Value', 'Wage', 'AT Apps', 'AT Gls', 'Team',
       'Caps', 'Yth Apps', 'Style', 'Rc Injury', 'Best Role', 'Best Duty',
       'Best Pos', 'Acceleration', 'Aerial Reach', 'Aggression', 'Agility',
       'Anticipation', 'Balance', 'Bravery', 'Command of Area',
       'Communication', 'Composure', 'Concentration', 'Corners', 'Crossing',
       'Decisions', 'Determination', 'Dribbling', 'Eccentricity', 'Finishing',
       'First Touch', 'Flair', 'Free Kick Taking', 'Handling', 'Heading',
       'Jumping Reach', 'Kicking', 'Leadership', 'Long Shots', 'Long Throws',
       'Marking', 'Fitness', 'Off the Ball', 'One vs One', 'Pace', 'Passing',
       'Penalty Taking', 'Positioning', 'Punching', 'Reflexes', 'Rushing Out',
       'Stamina', 'Strength', 'Tackling', 'Teamwork', 'Technique', 'Throwing',
       'Vision',

In [243]:
fm_df['Club_id'] = pd.Categorical(fm_df['Club']).codes
fm_df['League_id'] = pd.Categorical(fm_df['Division']).codes

In [244]:
fm_df = fm_df[fm_df['Inf'] != 'Yth']

In [245]:
fm_df = fm_df.rename(columns={"Best Pos": "Best_Pos"})

<br><br><br><br>
# Merged_DF

In [246]:
selected_leagues = ['English Premier Division',
 'Ligue 1 Uber Eats',
 'Spanish First Division',
 'Bundesliga',
 'Italian Serie A',
 'Eredivisie',
 'Portuguese Premier League',
 'Turkish Super League']

In [247]:
fm_df = fm_df[fm_df.Division.isin(selected_leagues)]

In [248]:
fm_df.columns

Index(['UID', 'Inf', 'Name', 'DoB', 'Nat', 'Division', 'Club', 'Based',
       'Preferred Foot', 'Right Foot', 'Left Foot', 'Position', 'Height',
       'Weight', 'Age', 'Transfer Value', 'Wage', 'AT Apps', 'AT Gls', 'Team',
       'Caps', 'Yth Apps', 'Style', 'Rc Injury', 'Best Role', 'Best Duty',
       'Best_Pos', 'Acceleration', 'Aerial Reach', 'Aggression', 'Agility',
       'Anticipation', 'Balance', 'Bravery', 'Command of Area',
       'Communication', 'Composure', 'Concentration', 'Corners', 'Crossing',
       'Decisions', 'Determination', 'Dribbling', 'Eccentricity', 'Finishing',
       'First Touch', 'Flair', 'Free Kick Taking', 'Handling', 'Heading',
       'Jumping Reach', 'Kicking', 'Leadership', 'Long Shots', 'Long Throws',
       'Marking', 'Fitness', 'Off the Ball', 'One vs One', 'Pace', 'Passing',
       'Penalty Taking', 'Positioning', 'Punching', 'Reflexes', 'Rushing Out',
       'Stamina', 'Strength', 'Tackling', 'Teamwork', 'Technique', 'Throwing',
       'Vision',

In [249]:
for leg in selected_leagues:
    q = fm_df.query(f"Division == '{leg}'")["Club"].value_counts()
    print(leg , "------", len(q.to_list()))

English Premier Division ------ 20
Ligue 1 Uber Eats ------ 20
Spanish First Division ------ 20
Bundesliga ------ 18
Italian Serie A ------ 20
Eredivisie ------ 18
Portuguese Premier League ------ 18
Turkish Super League ------ 19


In [250]:
fm_df.query(f"Division == 'Turkish Super League'")["Club"].value_counts()

Club
Kayserispor         41
Beşiktaş            41
Trabzonspor         40
Adana D.S.          39
Fenerbahçe          37
Ankaragücü          37
İstanbulspor        36
Galatasaray         35
Hatayspor           35
Kasımpaşa           35
Antalyaspor         34
Alanyaspor          34
Başakşehir FK       34
Sivasspor           33
Giresunspor         33
Gaziantep FK        32
Fatih Karagümrük    32
Ümraniyespor        32
Konyaspor           31
Name: count, dtype: int64

In [251]:
# fm_df.query(f"Club == 'Adana D.S.'").head()

<br><br><br><br>
# Team Names Converter from fminside.com

In [252]:
url_df = fm_df.drop_duplicates(subset=["Club"], keep="first")
url_df.loc[:, 'UID'] = url_df['UID'].apply(lambda text: str(text))
url_df = url_df[["Club", "UID"]]

In [253]:
def Player_to_URL(player_id):
    slug = player_id + "-x-x"
    return f"https://fminside.net/players/3-fm-23/{slug}"

In [254]:
def Convert_Team_Name_From_URL(player_id, club):
    url = Player_to_URL(player_id)
    req = requests.get(url)
    html = req.content
    soup = BeautifulSoup(html, "lxml")
    club_name_element = soup.find('div', class_='meta').find('ul').find_all('li')[0].text
    club_name = club_name_element.strip()
    return {"fm":club,"normal":club_name}

In [255]:
team_mapping_array = []
raw_names = url_df.to_dict()
uids = url_df["UID"]
clubs = url_df["Club"]
clubs_array = clubs.to_list()

In [256]:
# for uid,club in zip(uids,clubs):
#     new_entry = Convert_Team_Name_From_URL(uid, club)
#     print(new_entry)
#     team_mapping_array.append(new_entry)

In [257]:
fm_df.query("Name == 'Joshua Kimmich'")

UID Inf            Name         DoB  Nat    Division       Club  \
42332  92039023      Joshua Kimmich  1995-02-08  GER  Bundesliga  FC Bayern   

                      Based Preferred Foot   Right Foot  ... Stamina Strength  \
42332  Germany (Bundesliga)          Right  Very Strong  ...    99.0    54.45   

      Tackling Teamwork  Technique Throwing Vision Work Rate Club_id League_id  
42332     59.4    84.15       79.2     4.95   89.1     94.05    3704       185  

[1 rows x 76 columns]

In [258]:
fminside_normal_names = pd.read_csv("players_db/fm23/fminside_team_names.csv")
# merged_df = pd.merge(fm_df, fminside_normal_names, how='inner', left_on='Club', right_on='fm')
# merged_df['Club'] = merged_df['normal'].where(merged_df['Club'] == merged_df['fm'], merged_df['Club'])
# # merged_df.drop(['fm', 'normal'], axis=1, inplace=True)
# merged_df = merged_df[merged_df.fm.isin(clubs_array)]

In [259]:
fminside_club_names = []
for index, row in fminside_normal_names.iterrows():
  x_fm = row["fm"]
  x_normal = row["normal"]
  club_dict = {"fm": x_fm, "normal": x_normal}
  fminside_club_names.append(club_dict)

In [260]:
def Rename_Club(club_name):
    matched_club = ""
    for f in fminside_club_names:
        if f["fm"] == club_name:
            return f["normal"]

In [261]:
fm_df['Club'] = fm_df["Club"].apply(lambda x: Rename_Club(x))

In [262]:
fm_df.query("Name == 'Joshua Kimmich'")

UID Inf            Name         DoB  Nat    Division       Club  \
42332  92039023      Joshua Kimmich  1995-02-08  GER  Bundesliga  FC Bayern   

                      Based Preferred Foot   Right Foot  ... Stamina Strength  \
42332  Germany (Bundesliga)          Right  Very Strong  ...    99.0    54.45   

      Tackling Teamwork  Technique Throwing Vision Work Rate Club_id League_id  
42332     59.4    84.15       79.2     4.95   89.1     94.05    3704       185  

[1 rows x 76 columns]

<br><br><br><br>
# Position Styling

In [263]:
position_mapping = {
    'M (C)': 'MC',
    'D (C)': 'DC',
    'GK': 'GK',
    'ST (C)': 'ST',
    'AM (L)': 'AML',
    'AM (R)': 'AMR',
    'D (R)': 'DR',
    'D (L)': 'DL',
    'DM': 'DM',
    'AM (C)': 'AMC',
    'M (L)': 'ML',
    'M (R)': 'MR',
    'WB (L)': 'WBL',
    'WB (R)': 'WBR'
}

In [264]:
def remove_parentheses(text):
    return text.replace("(", "").replace(")", "")

In [265]:
def pos_map(position):
    return position_mapping[position]

In [266]:
fm_df['Best_Pos'].value_counts()

Best_Pos
M (C)     977
D (C)     938
AM (L)    738
GK        704
ST (C)    648
AM (R)    555
D (R)     521
D (L)     475
DM        388
AM (C)    180
M (L)     130
M (R)     116
WB (R)     29
WB (L)     23
Name: count, dtype: int64

In [267]:
fm_df['Position'].value_counts()

Position
D (C)                    749
GK                       704
DM, M (C)                600
ST (C)                   519
M/AM (C)                 279
                        ... 
D (R), WB/M (RL)           1
D (R), WB (RL), M (R)      1
D (RLC), WB/M (R)          1
WB/M (RL)                  1
D/AM (R)                   1
Name: count, Length: 224, dtype: int64

In [268]:
def Multiple_Position_Beautifier(position_value):
    x = ""
    positions = position_value.split(",")
    for p in positions:
        p = p.strip()
        if p in position_mapping.keys():
            x += f"{position_mapping[p]},"
        elif "/" in p:
            slashed = p.split(" ")
            locations = slashed[0].split("/")
            sides = list(remove_parentheses(slashed[1]))
            for loc in locations:
                for side in sides:
                    splitted_pos = loc+side
                    x += f"{splitted_pos},"
        else:
            slashed = p.split(" ")
            loc = slashed[0]
            sides = list(remove_parentheses(slashed[1]))
            for side in sides:
                splitted_pos = loc+side
                x += f"{splitted_pos},"
    return x[:-1]

In [269]:
fm_df['Position'] = fm_df['Position'].apply(lambda row: Multiple_Position_Beautifier(row))
fm_df['Best_Pos'] = fm_df['Best_Pos'].apply(lambda row: Multiple_Position_Beautifier(row))

In [270]:
position_order = ['GK', 'WBR', 'DR', 'DC', 'DL', 'WBL', 'DM',  
                  'MC', 'MR', 'AMC', 'ML', 'AMR', 'ST', 'AML']
def get_position_rank(position):
    return position_order.index(position) if position in position_order else len(position_order)
fm_df['Pos_Rank'] = fm_df['Best_Pos'].apply(get_position_rank)

In [271]:
fm_df.query("Club == 'Chaves'")[["Best_Pos"]]

Best_Pos
168395       DC
168409       DC
168573      AML
168584       ST
168616      AML
168825      AML
169038      AMR
169074       GK
169161       MC
169202       ML
169207      AMC
169215       DR
169352       ST
169530      AML
169540      AML
169732       MC
169971       DM
170215       DL
170250      AMR
170287      AMR
170296      WBR
170327       ST
171625       MC
173209       GK
173950      AMR
177476      AMR
178013       DC
183728       DC

In [272]:
fm_df['Position'].value_counts()

Position
DC                  749
GK                  704
DM,MC               600
ST                  519
MC,AMC              279
                   ... 
DR,WBR,WBL,MR,ML      1
DR,WBR,WBL,MR         1
DR,DL,DC,WBR,MR       1
WBR,WBL,MR,ML         1
DR,AMR                1
Name: count, Length: 224, dtype: int64

In [273]:
fm_df['Best_Pos'].value_counts()

Best_Pos
MC     977
DC     938
AML    738
GK     704
ST     648
AMR    555
DR     521
DL     475
DM     388
AMC    180
ML     130
MR     116
WBR     29
WBL     23
Name: count, dtype: int64

<br><br><br><br>
# Players' Overall

- Keeper -> gk
- Defending -> def 
- Passing -> pas
- Dribbling -> dri
- Finishing -> fin
- Stamina -> sta
- Strength -> str
- Heading -> hed
- Mental -> men
- Intelligence -> iq

In [274]:
rating_attrs = ["gk","def","pas","dri","fin","sta","str","hed","men","iq"]

In [5]:
mental_attributes = [ "Aggression", "Anticipation", "Bravery", "Composure", "Concentration", "Decisions", "Determination", "Flair", "Leadership", "Off the Ball", "Positioning", "Teamwork", "Vision", "Work Rate" ]
physical_attributes = [ "Acceleration", "Agility", "Balance", "Jumping Reach", "Fitness", "Pace", "Stamina", "Strength" ]
technical_attributes = [ "Corners", "Crossing", "Dribbling", "Finishing", "First Touch", "Free Kick Taking", "Heading", "Long Shots", "Long Throws", "Marking", "Passing", "Penalty Taking", "Tackling", "Technique" ]

In [275]:
def Dict_Generator(attrs):
    weights = (len(attrs)*"0.1,").split(",")[:-1]
    return {key: float(weights[i]) for i, key in enumerate(attrs)}

<h3 style="color:orange;">  Keeper</h3>

In [276]:
current_ability = "gk"
fm_df[current_ability] = fm_df[gk_attributes + gk_technical_attributes].mean(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  gk
167966  Marc-André ter Stegen  75
151647                Alisson  74
42335            Manuel Neuer  73
151645                Ederson  73
167963              Jan Oblak  72

<h3 style="color:orange;">  Defending</h3>

In [7]:
weights =   {'Positioning': 0.35,
             'Heading': 0.05,
             'Marking': 0.30,
             'Tackling': 0.35}.values()
expert_attributes = list(weights.keys())

current_ability = "def"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

ValueError: Unable to coerce to Series, length must be 13: given 4

<h3 style="color:red;">  Passing</h3>

In [279]:
weights =   {'Crossing': 0.10,
             'First Touch': 0.10,
             'Passing': 0.35,
             'Technique': 0.20,
             'Decisions': 0.05,
             'Determination': 0.05,
             'Vision': 0.15}
expert_attributes = list(weights.keys())

current_ability = "pas"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  pas
167956     Lionel Messi   93
167959      Luka Modrić   92
151638  Kevin De Bruyne   89
167960    Karim Benzema   89
167970       Toni Kroos   88

<h3 style="color:red;">  Dribbling</h3>

In [280]:
weights =   {'Dribbling': 0.35,
             'First Touch': 0.05,
             'Acceleration': 0.20,
             'Agility': 0.10,
             'Balance': 0.10,
             'Fitness': 0.05,
             'Pace': 0.15}
expert_attributes = list(weights.keys())

current_ability = "dri"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  dri
167957    Kylian Mbappé   89
151738     Adama Traoré   86
167956     Lionel Messi   85
151640    Mohamed Salah   85
167964  Vinícius Júnior   84

<h3 style="color:red;">  Finishing</h3>

In [281]:
weights =   {'Finishing': 1}.values()
expert_attributes = list(weights.keys())

current_ability = "fin"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  fin
151646   Cristiano Ronaldo   94
73550   Zlatan Ibrahimović   94
167958  Robert Lewandowski   94
151641          Harry Kane   94
73547        Ciro Immobile   89

<h3 style="color:green;">  Stamina</h3>

In [282]:
weights =   {'Stamina': 1}.values()
expert_attributes = list(weights.keys())

current_ability = "sta"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  sta
42332          Joshua Kimmich   99
151729      James Ward-Prowse   99
73553        Marcelo Brozović   99
151691            Mason Mount   99
151758  Pierre-Emile Højbjerg   99

<h3 style="color:green;">  Strength</h3>

In [283]:
weights =   {'Strength': 1}.values()
expert_attributes = list(weights.keys())

current_ability = "str"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  str
151713           Kurt Zouma   99
73542         Romelu Lukaku   99
152008       Fraser Forster   94
151685          Reece James   94
151812  Aleksandar Mitrović   94

<h3 style="color:green;">  Heading</h3>

In [284]:
weights =   {'Heading': 1}.values()
expert_attributes = list(weights.keys())

current_ability = "hed"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  hed
167965       Sergio Ramos   99
73561      Olivier Giroud   94
75806         Milan Đurić   94
151646  Cristiano Ronaldo   94
167994     Edinson Cavani   89

<h3 style="color:purple;">  Mental</h3>

In [285]:
weights =   {'Aggression': 0.15,
             'Composure': 0.40,
             'Concentration': 0.45}
expert_attributes = list(weights.keys())

current_ability = "men"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  men
151660     Thiago Silva   90
167993      Axel Witsel   86
167959      Luka Modrić   84
167980  Sergio Busquets   84
151645          Ederson   84

<h3 style="color:purple;">  Intelligence</h3>

In [286]:
weights =   {'Vision': 0.35,
             'Anticipation': 0.20,
             'Decisions': 0.25,
             'Flair': 0.20}
expert_attributes = list(weights.keys())

current_ability = "iq"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  iq
167956     Lionel Messi  93
151638  Kevin De Bruyne  86
168016            Pedri  85
105366       Mesut Özil  85
167988   Marco Verratti  84

<h1 style="color:red;">  🥽TPR</h1>

In [287]:
positions = fm_df['Best_Pos'].unique().tolist()

In [288]:
def TPR_by_Position(row, pos=False):
    if not pos:
        position = row["Best_Pos"]
    else:
        position = pos
        
    rating_weights = {}
    if position == "GK":
        rating_weights = { "gk": 0.95, "def": 0, "pas": 0, "dri": 0, "fin": 0, "sta": 0, "str": 0, "hed": 0, "men": 0.05, "iq": 0 }
    elif position == "DC":
        rating_weights = { "gk": 0, "def": 0.25, "pas": 0.12, "dri": 0, "fin": 0, "sta": 0.19, "str": 0.15, "hed": 0.15, "men": 0.1, "iq": 0.05 }
    elif position == "DL" or position == "DR":
        rating_weights = { "gk": 0, "def": 0.22, "pas": 0.25, "dri": 0.23, "fin": 0, "sta": 0.17, "str": 0.04, "hed": 0.02, "men": 0.03, "iq": 0.04 }
    elif position == "WBL" or position == "WBR":
        rating_weights = { "gk": 0, "def": 0.09, "pas": 0.21, "dri": 0.32, "fin": 0, "sta": 0.22, "str": 0.04, "hed": 0.01, "men": 0.03, "iq": 0.08 }
    elif position == "DM":
        rating_weights = { "gk": 0, "def": 0.1, "pas": 0.23, "dri": 0.09, "fin": 0.02, "sta": 0.19, "str": 0.1, "hed": 0.04, "men": 0.08, "iq": 0.15 }
    elif position == "MC":
        rating_weights = { "gk": 0, "def": 0.06, "pas": 0.26, "dri": 0.09, "fin": 0.07, "sta": 0.18, "str": 0.05, "hed": 0.03, "men": 0.08, "iq": 0.18 }
    elif position == "ML" or position == "MR":
        rating_weights = { "gk": 0, "def": 0.01, "pas": 0.11, "dri": 0.2, "fin": 0.14, "sta": 0.17, "str": 0.11, "hed": 0.03, "men": 0.07, "iq": 0.09 }
    elif position == "AMC":
        rating_weights = { "gk": 0, "def": 0, "pas": 0.25, "dri": 0.19, "fin": 0.16, "sta": 0.06, "str": 0.04, "hed": 0, "men": 0.03, "iq": 0.27 }
    elif position == "AML" or position == "AMR":
        rating_weights = { "gk": 0, "def": 0, "pas": 0.08, "dri": 0.34, "fin": 0.22, "sta": 0.16, "str": 0.03, "hed": 0.01, "men": 0.05, "iq": 0.11 }
    elif position == "ST":
        rating_weights = { "gk": 0, "def": 0, "pas": 0.09, "dri": 0.14, "fin": 0.3, "sta": 0.12, "str": 0.1, "hed": 0.12, "men": 0.06, "iq": 0.07 }
    
    rating_attrs = list(rating_weights.keys())
    tpr = (row[rating_attrs] * pd.Series(rating_weights)).sum()
    return int(tpr)

In [301]:
fm_df["tpr"] = fm_df.apply(TPR_by_Position, axis=1)

for pos in positions:
    fm_df[f"tpr_{pos}"] = 0

In [303]:
# Standardize between 45-95
min_val = 45
max_val = 95
def standardize(column):
    min_col = column.min()
    max_col = column.max()
    col_standard = (column - min_col) / (max_col - min_col) * (max_val - min_val) + min_val
    return col_standard.astype(int)

fm_df["tpr"] = standardize(fm_df["tpr"])

In [304]:
posdf = pd.read_csv("players_db/fm23/position_regression_formulas.csv")

In [305]:
def Position_Regression(row, pos):
    best_pos = row.Best_Pos
    x = row.tpr
    m = posdf[posdf.Best_Pos == best_pos][f"{pos}_slope"]
    b = posdf[posdf.Best_Pos == best_pos][f"{pos}_intercept"]
    y = int(m*x + b)
    return max(y, 0) 

In [306]:
for pos in positions:
    fm_df[f"tpr_{pos}"] = fm_df.apply(lambda row: TPR_by_Position(row, pos) if pos == row.Best_Pos or pos in row.Position else Position_Regression(row, pos), axis=1)

In [307]:
fm_df.sort_values("tpr",ascending=False)[["Name","tpr"]].head(n=10)

Name  tpr
167956        Lionel Messi   95
167958  Robert Lewandowski   93
151641          Harry Kane   91
151640       Mohamed Salah   90
151639      Erling Haaland   90
151638     Kevin De Bruyne   90
167957       Kylian Mbappé   90
151644     Virgil van Dijk   90
151642          Rúben Dias   89
73550   Zlatan Ibrahimović   88

In [308]:
fm_df.query("Name in 'Lionel Messi'").iloc[0].to_dict()

{'UID': 7458500,
 'Inf': '',
 'Name': 'Lionel Messi',
 'DoB': '1987-06-24',
 'Nat': 'ARG',
 'Division': 'Ligue 1 Uber Eats',
 'Club': 'Paris Saint-Germain',
 'Based': 'France (Ligue 1 Uber Eats)',
 'Preferred Foot': 'Left',
 'Right Foot': 'Fairly Strong',
 'Left Foot': 'Very Strong',
 'Position': 'AMR,AMC,ST',
 'Height': '169',
 'Weight': '67 kg',
 'Age': 35,
 'Transfer Value': '€50M - €76M',
 'Wage': '€3,366,000 p/m',
 'AT Apps': '578',
 'AT Gls': '491',
 'Team': 'Main',
 'Caps': 162,
 'Yth Apps': '18',
 'Style': 'Creative',
 'Rc Injury': '-',
 'Best Role': 'Attacking Midfielder',
 'Best Duty': 'Attack',
 'Best_Pos': 'AMC',
 'Acceleration': 74.25,
 'Aerial Reach': 9.9,
 'Aggression': 34.65,
 'Agility': 74.25,
 'Anticipation': 84.15,
 'Balance': 89.10000000000001,
 'Bravery': 49.5,
 'Command of Area': 9.9,
 'Communication': 4.95,
 'Composure': 79.2,
 'Concentration': 54.45,
 'Corners': 74.25,
 'Crossing': 74.25,
 'Decisions': 89.10000000000001,
 'Determination': 99.0,
 'Dribbling': 99.

# Matching fbref Player Names with FM Player Names

In [309]:
from rapidfuzz import process
def find_best_match(name, choices):
    return process.extractOne(name, choices)

In [310]:
top5_leagues = ['Ligue 1 Uber Eats', 'English Premier Division', 'Italian Serie A',
       'Spanish First Division', 'Bundesliga'] 

In [311]:
url = "https://fbref.com/en/comps/Big5/2022-2023/passing/players/2022-2023-Big-5-European-Leagues-Stats"

In [312]:
data = pd.read_html(url)

In [313]:
data[0].columns = data[0].columns.get_level_values(1)

In [314]:
data[0].head()

Rk            Player   Nation    Pos           Squad                Comp  \
0  1  Brenden Aaronson   us USA  MF,FW    Leeds United  eng Premier League   
1  2   Paxten Aaronson   us USA  MF,DF  Eint Frankfurt       de Bundesliga   
2  3    James Abankwah   ie IRL     DF         Udinese          it Serie A   
3  4     George Abbott  eng ENG     MF       Tottenham  eng Premier League   
4  5  Yunis Abdelhamid   ma MAR     DF           Reims          fr Ligue 1   

  Age  Born   90s   Cmp  ... Ast  xAG   xA A-xAG  KP  1/3 PPA CrsPA PrgP  \
0  21  2000  26.4   592  ...   3  4.2  2.6  -1.2  46   47  16     4   86   
1  18  2003   1.9    51  ...   0  0.0  0.1   0.0   1    3   0     0    6   
2  18  2004   0.7    23  ...   0  0.0  0.0   0.0   0    0   0     0    0   
3  16  2005   0.0     1  ...   0  0.0  0.0   0.0   0    0   0     0    0   
4  34  1987  37.0  1679  ...   2  1.0  0.9  +1.0  13  155   5     0  215   

   Matches  
0  Matches  
1  Matches  
2  Matches  
3  Matches  
4  Matches  

[5 rows x 33 columns]

In [315]:
# fm_df = pd.read_csv("players_db/fm23/fm23db_processed.csv")
fbref_players = data[0]["Player"].tolist()
fm_players = fm_df[fm_df.Division.isin(top5_leagues)]["Name"].tolist()

In [316]:
for player_name in fm_players:
    answer, score, other = find_best_match(player_name, fbref_players)
    where_id = fm_df[fm_df.Name==player_name].iloc[0].name
    fm_df.at[where_id, 'fbref_player_name'] = answer

# CSV EXPORT

In [317]:
fm_df.to_csv("players_db/fm23/fm23db_processed.csv", index=False)